Version 0.0.2.9.2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

# Ereignisse
-------------------------

Im Folgenden soll teilweise abgedeckt durch ein Kolloquium der Ursprung der im Weiteren verwendeten Datensätze näher betrachtet werden $-$ der CMS-Detektor. Es soll der Aufbau des Detektors und die Funktionsweise einzelner Detektorkomponenten besprochen werden. Zudem sollen ebenfalls Ereignisse betrachtet werden, anhand derer wichtige physikalische Größen grafisch dargestellt werden können. Für beide Fälle kann das WebInterface [Ispy-WebGL](https://github.com/cms-outreach/ispy-webgl) verwendet werden.

In [ ]:
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-info">
Identifizieren und benennen Sie alle Zerfälle aus der lokalen Sammlung und halten Sie die Darstellungen einiger davon fest.
</div>

Mit einer Internetverbindung:

In [ ]:
%%html
<iframe src="https://ispy-webgl.web.cern.ch/ispy-webgl/" width="100%" height="700"></iframe>

Ohne eine Internetverbindung: Lokal die `index.html` aus dem [Github Repository](https://github.com/cms-outreach/ispy-webgl) in einem Webbrowser öffnen.

Im Weiteren werden nur Ereignisse aus dem Zerfall in vier Leptonen (Myonen und Elektronen, keine Tau-Leptonen) betrachtet.

# Datenformat
----------------------

Das im Weiteren verwendete Datenformat, aus dem alle notwendigen Größen genommen werden ist ein modifiziertes .CSV Format. Der Vorteil dieses Formates ist, dass immer und zu jeder Zeit die Daten so wie sie auch ein Mensch lesen würde betrachten werden können.

Der Abschnitt dient dazu sich mit dem Datenformat vertraut zu machen um die Aufgabe in dem nachfolgendem Abschnitt durchführen zu können.

Die Trennung einzelner Variablen in einem Event geschieht mithilfe von ";". Die Einträge der einzelnen Leptonen innerhalb eines Events werden dagegen klassisch mit "," getrennt. Damit entsteht der Vorteil einer individuellen Anzahl an Leptonen in einem Event ohne die Einführung von zusätzlichen Platzhaltern.

In [ ]:
%matplotlib inline
#%matplotlib qt
import numpy as np
import pandas as pd

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

name_1 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_2el2mu_init.csv"
name_2 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_4mu_init.csv"

dataframe_1 = pd.read_csv(name_1, delimiter=";")
dataframe_2 = pd.read_csv(name_2, delimiter=";")

In [ ]:
dataframe_1

In [ ]:
dataframe_2

Für die Bearbeitung einzelner Größen können die jeweiligen Elemente aus dem String (`str`) Datenformat wieder zurück in einer Liste (`list`) umgewandelt werden:
Hierzu stehen die Optionen offen entweder die Standardbibliothek `ast`

In [ ]:
import ast
px = ast.literal_eval(f"[{dataframe_2.loc[3, 'px']}]")
px = np.array(px, dtype=float)
px

oder die `split` Methode für die `str` zu vernwenden

In [ ]:
py = dataframe_2.loc[3, 'py'].split(",")
py = np.array(py, dtype=float)
py

Daraus können neue Größen wie der transversale Impuls bestimmt werden anhand dessen eine Eingrenzung in den Größen getroffen werden kann und soll

In [ ]:
pt = np.sqrt(px ** 2 + py ** 2)
pt

Für Myonen muss ein Minimalwert von 5 GeV für den Transversalimpuls überschritten werden. Für Transversalimpuls Werte darunter steigt die Wahrscheinlichkeit einer Missidentifikation der Myonen an.
Insofern müssen alle Myonen in diesem Ereignis, die die Bedingung nicht erfüllen, verworfen werden.

In [ ]:
pt_minimum_filter = pt > 5
pt_minimum_filter

In [ ]:
pt = pt[pt_minimum_filter]
pt

# Anwendung der Filter mit Hilfe von Apply
----------------------

Die Anwendung eines Filters erfolgt auf alle Variablen innerhalb eines Ereignisses und ist von dem Filter für das nächste Ereignis verschieden.
Die reihenweise Anwendung des Filters auf den Datensatz ist bereits implementiert und über die Apply Klasse anwendbar.

In [ ]:
from include.processing.Apply import Apply

Bei der Erstellung des Objekts muss zusätzlich eine "allowed" und eine "calculation" Klasse übergeben werden. Die bei der Anwendung der Filter notwendigen Größen oder Filter werden anhand dieser Klassen berechnet.
Für des Beispiel des minimalen Transversalimpulses lassen sich die beiden Klassen als eine Ansammlung von Funktionen darstellen, die alleine stehen könnten (@staticmethod), aber logisch in einer Klasse zusammengefasst werden.

In [ ]:
class Calc_Start(object):
    
    @staticmethod
    def pt(px, py):
        return np.sqrt(px ** 2 + py ** 2)

class Allowed_Start(object):
    
    @staticmethod
    def min_pt(pt, look_for):
        return pt > 7.0 if look_for == "electron" else (
            pt > 5.0 if look_for == "muon" else None)

In [ ]:
print(name_2)
process = Apply(input_=name_2, particle_type="muon", use_n_rows=10000,
                calc_instance=Calc_Start, allowed_instance=Allowed_Start)

In dem Datensatz `name_2` existiert die Größe des Transversalimpulses noch nicht. In einem Zwischenschritt, der in dem Filter `"check_min_pt"` schon enthalten ist lässt sich der Transversalimpuls auch explizit hinzufügen:

In [ ]:
process.add_variable("pt")
process.data

Die Häufigkeitsverteilung einer solchen Variable kann durch die Methode `hist_of_variable` dargestellt werden.

In [ ]:
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Die Anwendung des Filters für den minimalen Transversalimpuls und die Entfernung aller nachfolgend entstandenen Ereignisse, die weniger als vier Leptonen enthalten liefert die entsprechende Verteilung:

In [ ]:
process.filter(filter_name="check_min_pt")
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Ebenso lassen sich auch andere Variablen durch die Häufigkeitsverteilungen visualisieren.
Der durchgeführte Cut beeinflusst hauptsächlich die Verteilung des Transversalimpulses, ändert an der Verteilung anderer Größen dagegen wenig.

<div class="alert alert-info">
Betrachten Sie die Verteilungen einiger in den Datensätzen vorhandenen Größen.

</div>


In [ ]:
# here goes the code

# Berechnung wichtiger Größen
----------------------

<div class="alert alert-info">

Dem Beispiel des Transversalimpulses folgend implementieren sie alle nachfolgend notwendigen Größen und visualisieren sie angemessen die
  * Pseudorapidität $\eta$
  * Transversalimpuls $p_T$
  * Azimutalwinkel $\phi$

Und erklären sie ihre Beobachtungen.

(Überprüfen Sie ihre Implementierungen mithilfe der MC-Simulationen des Untergrundes.)
</div>

Der dafür verwendete Klassenskelett erbt die vorhergehende Methode für den Transversalimpuls und einer Initialklasse, die Methoden wie die Rekonstruktion der Z-Bosonen Paare enthält.

In [ ]:
class CalcStudent(Calc_Start):
    '''
    Class for the calculation of certain sizes that are used for
    the cuts or are essential for the reconstruction.
    '''

    @staticmethod
    def combined_charge(charge, combine_num):
        '''
        Tests whether an electrically neutral charge combination is possible.

        :param charge: ndarray
                       1D array containing data with "int" type.
        :param combine_num: int
                            4 if look_for is not "both", 2 else
        :return: bool
        '''
        # code

    @staticmethod
    def eta(px=None, py=None, pz=None, energy=None):
        '''
        Calculates the pseudorapidity.
        Optional with or without energy.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                    1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def invariant_mass_square(px, py, pz, energy=None, eta=None, phi=None):
        '''
        Calculates the square of the invariant mass.
        Optional with or without energy.
        Optionally with or without eta and phi.

        :param phi: ndarray
                    1D array containing data with "float" type.
        :param eta: ndarray
                    1D array containing data with "float" type.
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                   1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def phi(px, py):
        '''
        Calculation of the angle phi.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py:  ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def delta_phi(phi1, phi2):
        '''
        Calculation of the difference between two phi angles.

        :param phi1: ndarray
                     1D array containing data with "float" type.
        :param phi2: ndarray
                     1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def delta_r(eta, phi):
        '''
        Calculation of delta_r.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param phi: ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

# Erstellung der Cuts
----------------------

Analog zu dem Cut auf den minimalen Transversalimpuls lassen sich weitere Cuts einführen, die weiter verwendet werden. Ein Beispiel welche Cuts seitens CMS hierfür verwendet wurden kann in der [offiziellen Veröffentlichung](https://arxiv.org/pdf/1207.7235.pdf) nachgelesen werden.

<div class="alert alert-info">

Implementieren sie die in der Klasse 'AllowedStudent' aufgelisteten Methoden. Schätzen sie die Wahl der Cuts mithilfe von den Verteilungen im Kapitel **Anwendung der Filter mithilfe von Apply**, dem **Detektor** und den **kinematischen Einschränkungen** der Ereignisse.

(Überprüfen sie ihre Implementierungen mithilfe der MC-Simulationen des Untergrundes.)
    
</div>

In [ ]:
class AllowedStudent(Allowed_Start):
    '''
    Class that introduces certain cuts and thus restricts the leptons in the events.
    '''
    
    @staticmethod
    def delta_r(delta_r):
        '''
        Checks if delta_r is smaller than the allowed value.

        :param delta_r: ndarray
                        1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code

    @staticmethod
    def rel_pf_iso(rel_pf_iso):
        '''
        Checks if rel_pf_iso is smaller than the allowed value.

        :param rel_pf_iso: ndarray
                           1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code
        
    @staticmethod
    def misshits(misshits):
        '''
        Checks if the minimum number of misshits was kept.

        :param misshits:
        :return:
        '''
        # code
        
    @staticmethod
    def pt(p_t, look_for, coll_size=4):
        '''
        Checks if the exact pedingun regarding pt is observed.
        (>20 GeV: >= 1; >10 GeV: >= 2; >Minimum pt: >= 4).

        :param p_t: ndarray
                    1D array containing data with `float` type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :param coll_size: int
                          4 if look_for is not "both", 2 else
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code
        
    @staticmethod
    def eta(eta, look_for):
        '''
        Checks if the pseudorapidity of leptons is valid.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def lepton_type(typ, look_for):
        '''
        Checks for the permitted classification of leptons.

        :param typ: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def impact_param(sip3d, dxy, dz):
        '''
        Checks if the impact parameters of the collision are valid and sorts out
        events that do not have a clear and equal collision point.

        :param sip3d: ndarray
                      1D array containing data with "float" type.
        :param dxy: ndarray
                    1D array containing data with "float" type.
        :param dz: ndarray
                   1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def zz(z1, z2):
        '''
        Checks if the Z1 candidate and the Z2 candidate is within the allowed range.

        :param z1: float
        :param z2: float
        :return: bool
        '''
        # code

Kombination des implementieren Codes mit den teilweise zur Verfügung gestellten Klassen:

In [ ]:
from include.processing.CalcAndAllowerInit import AllowedInit
from include.processing.CalcAndAllowerInit import CalcInit

AllowedInit.a_allowed_instance = AllowedStudent
AllowedInit.a_calc_instancea = CalcStudent
class Allowed(AllowedStudent, AllowedInit):
    pass
    
CalcInit.c_allowed_instance = Allowed
CalcInit.c_calc_instance = CalcStudent
class Calc(CalcStudent, CalcInit):
    pass

# Anwendung der Filter und Rekonstruktion auf MC - Simulationen
--------------------------------------------------------------------------------------------------------------

Die vorhandenen Implementierungen der Cuts wurden mithilfe der Untergrund-MC-Simulationen getestet. Dies soll das gezielte Hinarbeiten auf ein gewisses Ziel (eigentliche Messung) verhindern, da durch eine geringe Anzahl an späteren Ereignissen in der eigentlichen Messung die Tatsache weitestgehend vermieden werden soll, gezielt Ereignisse in einem Bereich subjektiv auszuwählen.

<div class="alert alert-info">

Wenden Sie nun alle von ihnen implementierten Cuts final auf die MC-Simulationen und Messung an.
    
</div>

Dafür ist es sinnvoll sich eine Liste an Tupeln zu erstellen:

In [ ]:
from include.processing.ApplyHelper import ProcessHelper


# Alle Untergrund MC und Signal MC für m = 125 GeV
mc_files = True
# Durchgeführte Messung
run_files = True

# Speicherort der Messung
dir_measurement = "../data/for_long_analysis/ru_init/"
# Speicherort der Untergrund MC und 125 GeV Signal MC
dir_mc = "../data/for_long_analysis/mc_init/"


file_tuples = []
if mc_files:
    file_tuples += ProcessHelper.create_tuple(dir_mc)    

if run_files:
    file_tuples += ProcessHelper.create_tuple(dir_measurement)

Ein einzelner solcher `namedtuple` enthält die Datei und den Teilchentyp des Datensatzes (notwendig für `Apply`):

In [ ]:
file_tuples[0]

Der ganze Filter- und Rekonstruktionsablauf kann in einer Funktion zusammengefasst werden. Die Reihenfolge der Filter ist hierbei für die Laufzeit entscheidend.

Zur Auswahl stehen folgende Filter- und Rekonstruktionsschritte:

In [ ]:
Apply.help()

Die Funktion kann die folgende Form haben:

In [ ]:
def filter_and_reco_process(used_pair):
    process = Apply(input_=used_pair.name, 
                    particle_type=used_pair.particle, 
                    multi_cpu=True, use_swifter=False,
                    calc_instance=Calc, 
                    allowed_instance=Allowed)
    
    # Logische Reihenfolge Wählen
    # quicksave: Speichert den Datensatz NACH dem Anwenden 
    # des Filter- bzw Rekonstruktionsschrittes
    
    # bereits angewendet um die Reduktion der Datenmengen auf ein Notebook Niveau (1 Kern) zu ermöglichen. 
    # Typ nicht mehr im Datensatz enthalten!
    # process.filter(filter_name="check_type", quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftT"))
    
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_min_pt")
    process.filter(filter_name="check_impact_param")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_exact_pt")
    process.filter(filter_name="check_m_2l")
    process.filter(filter_name="check_rel_iso")
    process.filter(filter_name="check_q")
    if process.particle_type != "muon":
        process.filter(filter_name="check_misshit")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_eta")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_m_4l",
                   quicksave=ProcessHelper.change_on_affix(used_pair.name, "befZ"))
    process.reconstruct(reco_name="zz", 
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftZ"))
    process.reconstruct(reco_name="mass_4l_out_zz",
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftH"))
    del process

<div class="alert alert-info">

Wählen Sie die passende Reihenfolge der Filter- und Rekonstruktionsschritte.
    
(Um die Geschwindigkeit abzuschätzen, mit dem die einzelnen Filter auf einem Datensatz ausgeführt werden empfiehlt es sich den Datensatz der Untergrund-MC-Simulationen zu verwenden und bei der Initialisierung der `Apply`-Instanz folgende `kwargs` zu verwenden: `multi_cpu=False, use_swifter=True`)
    
</div>

Die Anwendung von `filter_and_reco_process` auf die Datensätze: 

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output

if input("Run all filter + reco (y/n): ") == "y":
    for pair in tqdm(file_tuples):
        filter_and_reco_process(pair)
        clear_output()

# Parametrisierung des Untergrundes
------------------------------------------------------------

Für die Vermeidung von Fluktuationen ist es vorteilhaft die Untergrundverteilung der MC-Simulationen durch eine Wahrscheinlichkeitsdichte auszudrücken.

<div class="alert alert-info">

Parametrisieren Sie den Untergrund durch ein geeignetes Modell. Stellen Sie zudem sicher, dass das gewählte Modell das passendste ist.
    
</div>

Die Implementierung von Linearkombinationen der Legendrepolynome ist bereits durchgeführt und in der Klasse `LLC` zusammengefasst. Ebenso können eigenständige Funktionen verwendet werden.

In [ ]:
import matplotlib.pyplot as plt
from include.parameterization.LegendrePoly import LegendreLinearCombination as LLC

In [ ]:
llc_background = LLC(x_mean=0.0) # Zentrierung um x_mean

x_ = np.linspace(-1, 1, 1000)
plt.plot(x_, llc_background.grade_0(x_, a=1))
plt.plot(x_, llc_background.grade_1(x_, a=1, b=1))
plt.plot(x_, llc_background.grade_2(x_, a=1, b=1, c=1))

Vorbereitend sollen die Binanzahl und das betrachtete Intervall gewählt werden:

In [ ]:
bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/"  
llc_background = LLC(x_mean=(999.1 + 999.2)/2.)

Variante:

 1. <ins>Die Daten aus dem Histogramm werden roh ausgegeben.</ins> 
<div class="alert alert-info">
     
    * Berechnen Sie die Unsicherheiten auf die einzelnen Bineinträge des skalierten Histogramms
    * Skalieren Sie die Histogramme der einzelnen Kanäle und fassen sie diese in einem gemeinsamen Histogramm zusammen.

    (Das Zugreifen auf die einzelnen Teile kann über den Attribut ´data´ geschehen, wobei das ein geschachteltes ´dict´ ist. Die erste Ebene enthält die drei einzelnen Kanäle. In jedem der Kanäle ist anschließend die unskalierten Histogrammeinträge, der Korrekturfaktor und die Daten, die für die Histogrammbildung verwendet wurden, gespeichert.)

</div>

In [ ]:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="background")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_ZZ_4el_2012"]

In [ ]:
import kafe2 as K2
# code

2. <ins>Die Daten aus dem Histogramm werden zusammengefasst ausgegeben</ins> 
    <div class="alert alert-info">
     
    Verwenden Sie die bereits zusammengefassten Größen für den Histogrammfit.

</div>

In [ ]:
from include.fits.McFitInit import McFitInit

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir, info=info)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

Der eigentliche Fit erfolgt mithilfe des in 'kafe2' implementiertem Histogramfits. Ein Beispiel für einen Histogrammfit findet sich [hier](https://kafe2.readthedocs.io/en/latest/parts/user_guide.html#example-5-histogram-fit)

Es ist vorteilhaft sich eine Funktion oder Klasse zu erstellen, um umständliche Wiederholungen zu vermeiden.

In [ ]:
import kafe2 as K2
# code

# Auswahl der passenden  Signal MC
-----------------------------------------------------------

Das Ziel dieses Abschnittes ist eine Eingrenzung der möglichen Signal-MC-Simulationen durchzuführen und sich auf ein einziges festzulegen und damit weiterzuarbeiten.

## Vorläufige Parametrisierung der Signal MC
--------------------------------------------------------------

Es stehen unterschiedliche Signal-MC Simulationen zur Verfügung. Diese wurden durch eine Gaußverteilung $\mathcal{G} \left( m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}} \right)$ grob parametrisiert. Das Ziel ist es die Abhängigkeit der Breite von der invarianten Masse festzustellen und durch ein passendes Modell zu beschreiben. Die Auflösung des Transversalimpulses des Detektors ist hierbei der größte Einfluss.


<div class="alert alert-info">

Wählen Sie die am besten passendste Signal-MC Simulation aus einer gegebenen Menge an Signal-MC Simulationen aus und begründen sie ihre Wahl.
</div>

Die Ergebnisse dieser Parametrisierungen sind in einem Pandas Dataframe zusammengefasst.

In [ ]:
import pandas as pd
df_ = pd.read_csv("../data/for_long_analysis/other_mc_gauss_param/gauss_params.csv")
df_
sigma, sigma_err, mu, mu_err = tuple(df_[["sigma", "sigma_err", "mu", "mu_err"]].values.T)

<div class="alert alert-info">

Parametrisieren Sie $\sigma_{\mathrm{G}}(m_{4\ell})$ anhand der vorhandenen Daten.
  
</div>

In [ ]:
def sigma_model(x, *args):
    """
    Model to for sigma parameterization
    
    """
    # code
    
    return x

# code

## Likelihood - Quotienten - Test
----------------------------------------------

Für die Auswahl zwischen den unterschiedlichen Simulationen wird explizit der Likelihood-Quotienten-Test verwendet.
Hierzu werden die Nullhypothese $H_0$, die nur dem Untergrund entspricht, und die Gegenhypothese $H_1$, die dem Untergrund und einem Signal entspricht, miteinander verglichen.

In die (extended) Likelihood der Hypothese $H_0$ fließt die Parametrisierung des Untergrundes $f_{\mathrm{U}}(x|\theta_{\mathrm{U}})$ ein wobei $f_{\mathrm{U}}$ das vorher festgelegte Modell aus der Untergrundparametrisierung mit den optimalen Parametern $\theta_{\mathrm{U}}$ ist.

Die für die Gegenhypothese verwendete Verteilung lässt sich beschreiben durch: $$f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) = \mu f_{\mathrm{S}}(m_{4\ell}|\theta_{\mathrm{S}}) + f_{\mathrm{U}}(m_{4\ell}|\theta_{\mathrm{U}}) \, , \\ \int_{\Omega} f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) \mathrm{d}m_{4\ell} = 1 $$ $f_{\mathrm{S}}(x|\theta_{\mathrm{S}})$ ist für diesen Fall eine Gaußverteilung $\mathcal{G}(m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}}(m_{\mathrm{H}} ))$ mit festgehaltenen Werten für den Schwerpunkt und die Breite. Für den Schätzer $\mu$ gilt: $$\mu \geq 0 \, .$$ Diese Einschränkung führt zu einer vernachlässigbaren Verzerrung und wird aus physikalischen Überlegungen eingeführt, da in der Verteilung der Invarianten Massen ein Überschuss an Ereignissen erwartet wird, wenn ein Higgs Boson vorhanden sein sollte.

Damit kann die zu betrachtende Größe – das Verhältnis aus den Likelihoods beider Hypothesen – gebildet werden. Für die numerische Stabilität wird der negative Logarithmus davon gebildet. Der Faktor zwei ist für die spätere asymptotische Form notwendig.

$$ q_0 = -2 \ln \left( \frac{\mathcal{L}(x|\mu = \hat{\mu})}{\mathcal{L}(x|\mu = 0)} \right) \, .$$ 
$\hat{\mu}$ ist hierbei der ideale Schätzer für eine feste Masse und Breite der Signalhypothese.

In [ ]:
lower_bound, upper_bound = hist_range
mass_array = np.linspace(lower_bound, upper_bound, 100)
sigma_model_arguments = tuple([])
sigma_array = sigma_model(mass_array, *sigma_model_arguments)

<div class="alert alert-info">

Implementieren Sie die den Quotienten Likelihood Test. Für die numerische Integration können Sie die Methode ['scipy.integrate.quad'](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) verwenden. Die Minimierung der Likelihood Funktion können sie mithilfe von ['iminuit'](https://nbviewer.jupyter.org/github/scikit-hep/iminuit/blob/master/tutorial/basic_tutorial.ipynb
) durchführen.

</div>

In [ ]:
# Messung:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 9999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/" 
ru_dir = "../data/for_long_analysis/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
data = raw_data.data
data

# Skalierung der MC-Simulationen an die Daten:
b_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
_, _, background_hist, _ = b_mc_d.variables_for_hist_fit
scale_b_mc = np.sum(background_hist)

s_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="signal", mc_dir=mc_dir)
_, _, signal_hist, _ = s_mc_d.variables_for_hist_fit
scale_s_mc = np.sum(signal_hist)

scale_s_mc, scale_b_mc

In [ ]:
# Iminuit Example:
from iminuit import Minuit

def extended_likelihood(mu=0.0):
    # FUnktion hat need to be minimized
    #code
    return mu

m = Minuit(extended_likelihood, 
           mu=0.5, error_mu=0.1, limit_mu=(0.0, 3.0),
           errordef=0.5)
m.migrad()
best_mu = m.values["mu"]


q0 = np.zeros(len(mass_array))

#code

Die asymptotische Form der Wahrscheinlichkeitsdichte von $q_0$ wenn kein Signal erwartet wird lässt sich beschreiben durch:
$$ f(q_0|\hat{\mu}=0) \rightarrow \frac{1}{2}\left( \delta (q_0) + \chi^2_{1} \right) \, .$$
daraus lässt sich der $p_0$ Wert berechnen nach:
$$ p_0 = \int\limits_{q_{0_{\mathrm{obs}}}}^{\infty} \mathrm{d}q_0 f(q_0|\hat{\mu}=0) \, . $$
aus diesem kann die Signifikanz abschätzend berechnen zu: $$Z = \Phi^{-1}(1-p_0) = \sqrt{q_{0_{\mathrm{obs}}}} \, .$$ In diesem Fall sollte diese jedoch nur als eine erste Abschätzung betrachtet werden da die eingehende Parametrisierung des Signals ungenau ist.
Ebenso wird an dieser Stelle die Systematik nicht betrachtet, es fließt nur die statistische Unsicherheit in die Berechnung von $p_0$ bzw. $Z$ ein.

Das Abspeichern geschieht in Form von einem Pandas Dataframe.

In [ ]:
q0, p0 = np.zeros(len(sigma_array)), np.zeros(len(sigma_array))
df = pd.DataFrame(data=np.array([mass_array, sigma_array, q0, p0, np.sqrt(q0)]).T,
                  columns=["mass_array", "gauss_sigma", "q0", "p0", "sqrt_q0"])

<div class="alert alert-info">

Visualisieren Sie die Ergebnisse. Für die Signifikanzniveaus kann die `get_p0_sigma_lines` aus `PlotHelper` verwendet werden.
</div>

In [ ]:
import matplotlib.pyplot as plt
from include.stattest.PlotHelper import PlotHelper

In [ ]:
fig, ax = plt.subplots(1,1)
PlotHelper.get_p0_sigma_lines(x_=mass_array, 
                             ax_obj_=ax, 
                             max_sigma_=4)
ax.plot(df.mass_array, df.p0+1)
plt.show()

# Betrachtung der finalen Verteilungen
------------------------------------------------------------------

Nach der Wahl der passenden Signalverteilung können final unterschiedliche Größen betrachtet werden.

<div class="alert alert-info">

Betrachten Sie die Verteilung der Vier-Leptonen-Invarianten Massen, sowie die Massen der beiden Z-Bosonen.
    
</div>

Des Weiteren sind auch folgende Größen darstellbar (wobei es fraglich ist, ob `z1_index`, `z2_index`, sowie `z1_tag` bzw. `z2_tag` Größen sind die eine ausführlicher Betrachtung bedürfen):

In [ ]:
from include.processing.ApplyHelper import ProcessHelper
print(ProcessHelper.print_possible_variables("../data/for_long_analysis/mc_aftH/MC_2012_H_to_ZZ_to_4L_2el2mu_aftH.csv"))

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="../data/for_long_analysis/mc_aftH",
           ru_dir="../data/for_long_analysis/ru_aftH", info=info)

In [ ]:
# Beispiel
h.variable("energy", 50, (0, 200))
ax = plt.gca()
ax.set_xlabel(r"$p_T$ in GeV")
ax.set_ylabel("Bineinträge")
plt.show()

In [ ]:
# weitere Verteilungen

# Parametrisierung der Signal MC
----------------------

Nach der Wahl der passenden Signalhypothese soll die Verteilung genauer parametrisiert werden. Dies geschieht analog zu der Untergrundparametrisierung.

<div class="alert alert-info">

Parametrisieren Sie die Signalverteilung durch ein geeignetes Modell. Stellen Sie zudem sicher, dass das gewählte Modell das passendste ist.

</div>

Zur Auswahl stehen den Studenten:
- Gaußverteilung
- Cauchyverteilung
- Voigt-Profil
- Single - Sided - Crystal - Ball
- Double - Sided - Crystal - Ball

In [ ]:
from include.parameterization.SignalFunction import SignalFunction as SF

Variante:

1. <ins>Die Daten aus dem Histogramm werden roh ausgegeben.</ins>
<div class="alert alert-info">

    * Berechnen Sie die Unsicherheiten auf die einzelnen Bineinträge des skalierten Histogramms
    * Skalieren Sie die Histogramme der einzelnen Kanäle und fassen sie diese in einem gemeinsamen Histogramm zusammen.

    (Das Zugreifen auf die einzelnen Teile kann über den Attribut data geschehen, wobei das ein geschachteltes `dict` ist. Die erste Ebene enthält die drei einzelnen Kanäle. In jedem der Kanäle ist anschließend die unskalierten Histogrammeinträge, der Korrekturfaktor und die Daten, die für die Histogrammbildung verwendet wurden, gespeichert.)

</div>

In [ ]:
bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/"

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="signal")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_H_ZZ_4el_2012"]

In [ ]:
import kafe2 as K2
# code

2. <ins>Die Daten aus dem Histogramm werden zusammengefasst ausgegeben</ins> 
    <div class="alert alert-info">
     
    Verwenden Sie die bereits zusammengefassten Größen für den Histogrammfit.

</div>

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

In [ ]:
import kafe2 as K2
# code

# Bestimmung der statistischen Signifikanz
----------------------

Nach der erfolgreichen Parametrisierung des Untergrundes und des Signals kann die Signifikanz des Signals bestimmt werden. Hierzu wird folgende Parametrisierung gewählt:
$$ f(x, \alpha_\mathrm{S}) = \alpha_\mathrm{S} f_{\mathrm{S}}(x|\theta_{\mathrm{S}}) + (1 - \alpha_\mathrm{S}) f_{\mathrm{U}}(x|\theta_{\mathrm{U}}) \, .$$

Der optimale Parameter $\alpha_{\mathrm{S}}$ entspricht der Signalstärke dieser Messung. Diese Bestimmung erfolgt mithilfe der eigenständig zu implementierenden $-2\ln\mathcal{L(\alpha_{\mathrm{S}})}$ - Funktion. Die zuvor bestimmten Parameter $\theta_{\mathrm{S}}$ und $\theta_{\mathrm{U}}$ der Wahrscheinlichkeitsdichte des Signals bzw. Untergrundes werden für die Bestimmung festgehalten (profiled Likelihood).

Die Betrachtung des Funktionswertes $-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)$ soll hierbei hervorgehoben werden. Dieser Entspricht nämlich der Annahme, dass in der vorhandenen Messung kein Signalanteil vorhanden ist und die Messung somit durch die Nullhypothese (nur Untergrund) erklärt werden kann. In asymptotischen Form lässt sich deshalb analog zu dem im Kapitel zur Parametrisierung der Signal MC Ausdruck die Signifikanz bestimmen mit: $$ Z = \sqrt{\frac{-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)}{\mathcal{L}_{\mathrm{min}}}} \, .$$

<div class="alert alert-info">
     
Implementieren Sie die $-2\ln\mathcal{L(\alpha_S, m_{4\ell})}$ Funktion (falls noch nicht vorhanden). Bestimmen Sie die Signalstärke zzgl. Unsicherheit der Messung. Geben Sie die Signifikanz an, mit der ein higgsartiges Boson gemessen wird. Geben Sie zudem die Masse des higgsartigen Bosons für die zweidimensionale Variante an und Vergleichen sie diese mit der Literatur. Visualisieren und diskutieren sie Ihre Ergebnisse.
    
Für die Minimierung kann wie bei QLT `iminuit` verwendet werden. Die numerische Integration kann wieder mithilfe von `scipy.integrate.quad` durchgeführt werden.
    
Ebenfalls ist es möglich einen `UnbinnedFit` mittels 'kafe2' durchzuführen und die extended Likelihood als Kostenfunktion selbstständig implementieren.
</div>

In [ ]:
# Messung:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/" 
ru_dir = "../data/for_long_analysis/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
raw_data.data

In [ ]:
# code iminuit

def extended_nll_iminuit(alpha_s, mass=125.0):
    # code
    return mass

In [ ]:
# code kafe2

def extended_nll_kafe2(data, model, parameter_values, parameter_constraints):
    """
    
    :param data: measurement array
    :param model: model array (calculate individual in # code)
    :param parameter_values:
    :param parameter_constraints: (not used)
    :return:
    """
    
    # check if one of the parameter_values is nan
    if any(f"{par}" == "nan" for par in parameter_values):
        return np.inf
    
    # your code
    
    _total_log_likelihood = 0.0 # your code
    
    # check if _total_log_likelihood is nan
    if np.isnan(_total_log_likelihood):
        return np.inf
    return -2.0 * _total_log_likelihood

## 1D profiled Likelihood
-----------------------------------

Die eindimensionale Variante variiert nur die Signalstärke. Die Masse wird nach der Parametrisierung festgehalten und nicht mehr variiert.

### Iminuit Variante

In [ ]:
# code (iminuit minimization)

Visualisierung:

In [ ]:
from include.stattest.PlotHelper import PlotHelper
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
alpha_ = np.linspace(0.0, 0.8, 1000)
nll_ = extended_nll(alpha_s=alpha_, mass=125)

# as example
nll_ = 150*(alpha_ - 0.4) ** 2

z_ = np.sqrt(nll_[0])

# code to further customize the plot

fig, ax = plt.subplots(1,1)
PlotHelper.plot_nll_scan_main_window(ax_obj_=ax, 
                                     x_=alpha_, 
                                     y_=nll_, 
                                     z_=z_)

ax_in = inset_axes(ax, width='50%', height='40%', loc="upper right")
PlotHelper.plot_nll_scan_inside_window(ax_obj_=ax_in, 
                                       x_=alpha_, y_=nll_, 
                                       x_ticks_=PlotHelper.get_error_points_from_nll(x_=alpha_, nll_=nll_))
mark_inset(ax, ax_in, loc1=2, loc2=4, fc="none", ec="grey", lw=0.5, alpha=0.5)
plt.show()

### kafe2 Variante

Für die `kafe2` Variante muss ein `UnbinnedContainer` erstellt und angepasst werden. Für die 1D Variante empfiehlt es sich die Methode `fix_parameter(par_name, value)` zu verwenden.
Die Signifikanz kann mithilfe der implementierten `extended_nll_kafe2` Kostenfunktion berechnet werden. Es ist empfehlenswert den Parameter `alpha` mittels `limit_parameter` auf einen sinnvollen Intervall zu beschränken.

In [ ]:
import kafe2 as K2

my_cost_function = K2.UnbinnedCostFunction_UserDefined(extended_nll_kafe2)

# more code

## 2D profiled Likelihood - Massenbestimmung
----------------------------------------------------------------

Abschließend wird auch die Masse bestimmt. Dies geschieht durch die zweidimensionale Variation – Masse und Signalstärke.

### Iminuit Variante

Ähnlich zu der 1D Variante kann auch gleichzeitig die beste Masse bestimmt werden. Das Vorgehen geschieht analog zu der 1D Variante/QLT Implementierung.

In [ ]:
# code:
#     - minimization
#     - plots

### kafe2 Variante

Der Unterschied  zu 1D Variante besteht nun lediglich darin, dass die Masse nicht mehr durch `fix_parameter` festgehalten wird, sondern in einem bestimmten Intervall (`limit_parameter`) best möglichst bestimmt wird.

In [ ]:
# code

## Kombination der Messungen: CMS und ATLAS
----------------------------------------------------------------

Um eine bessere Statistische Signifikanz zu erhalten ist es vom Vorteil weitere Messungen in die Gesamtbetrachtung hinzuzuziehen. Dies kann entweder durch die Hinzunahme weiterer Daten oder einem anderen Experiment erfolgen. Im Rahmen dieser Aufgabe wird das Zweite durchgeführt.

Für das ATLAS Experiment müssten deshalb ähnliche Schritte wie oben durchgeführt werden, wobei viele der Cuts jedoch Abweichungen aufweisen. Um die Arbeit etwas abzukürzen wird die finale Messung zur Verfügung gestellt, ebenso wie die Histogrammeinträge der MC Simulationen.

Zudem werden die Parameter der passendsten Funktionen (Linearkombination aus zentrierten Legendrepolynomen bis zur dritten Ordnung für den Untergrund und die DSCB Verteilung für das Signal) zur Verfügung gestellt (es ist jedoch ebenfalls möglich sich von diesen Tatsachen selbst zu überzeugen)

In [ ]:
import numpy as np
import pandas as pd
import kafe2 as K2
import os
from pprint import pprint

In [ ]:
atlas_data_dir_ = "../data/for_long_analysis/atlas_data_and_mc_hist/"
atlas_files = [os.path.join(atlas_data_dir_, file) for file in os.listdir(atlas_data_dir_) if ".csv" in file and "atlas" in file]
pprint(atlas_files)
hist_range_ = (106, 151)
num_bins = 45

data_A = np.loadtxt(atlas_files[2], delimiter=",")
data_A = data[(data >= hist_range_[0]) & (data <= hist_range_[1])]

mc_bac = np.loadtxt(atlas_files[1], delimiter=",").T
mc_bac_data = mc_bac[0]
mc_bac_error = mc_bac[1]

mc_sig = np.loadtxt(atlas_files[0], delimiter=",").T
mc_sig_data = mc_sig[0]
mc_sig_error = mc_sig[1]

In [ ]:
data_CMS = data
data_ATLAS = data_A

background_parameter_CMS = background_parameter
signal_parameter_CMS = signal_parameter

background_parameter_ATLAS  = np.array([0.025050083639971264, 
                                        0.00018744120591615378, 
                                        -1.178181555551107e-05, 
                                        1.1872474265844408e-07])
signal_parameter_ATLAS = np.array([2.1395802886491615, 
                                   124.19274632605674, 
                                   0.8324353801486924, 
                                   1.8034218165656613, 
                                   14.109501070824912, 
                                   19.9999999999894])
bfu_ATLAS = llc_background.grade_3
sfu_ATLAS = sf.DSCB

<div class="alert alert-info">
Kombinieren Sie die beiden Messungen und bestimmten sie die aus der Kombination resultierende Masse des Higgs Bosons, die die Signalwahrscheinlichkeit, sowie die Signifikanz, einer Messung desHiggs Bosons.

Sie können die von Ihnen bereits implementierte Funktionen weiterverwenden/modifizieren.
    
Hinweis:
    $$ \mathcal{L_{\mathrm{tot}}(\alpha_S, m_{4\ell})} = \mathcal{L_{1}(\alpha_S, m_{4\ell})} \mathcal{L_{2}(\alpha_S, m_{4\ell})} $$
</div>

In [ ]:
# code (1D and 2D minimization)
